In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv

# Set up the WebDriver (make sure to specify the correct path to your WebDriver)
driver = webdriver.Chrome()  # or webdriver.Firefox() for Firefox

# Navigate to the Twitter account
driver.get("https://x.com/vitalikbuterin")

# Wait for the page to load
time.sleep(5)  # Adjust sleep time as necessary

# Scroll to load more tweets
scroll_pause_time = 2
last_height = driver.execute_script("return document.body.scrollHeight")

tweets = []

while True:
    # Find all tweet elements
    tweet_elements = driver.find_elements(By.XPATH, '//article[@role="article"]')
    
    for tweet in tweet_elements:
        # Extract tweet text
        tweet_text = tweet.text
        tweets.append(tweet_text)

    # Scroll down to load more tweets
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)

    # Calculate new scroll height and compare with last height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Save tweets to a CSV file
with open('vitalik_tweets.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Date', 'Tweet', 'Likes', 'Retweets', 'Quote Tweets'])  # Header

    for tweet in tweets:
        # Split the tweet text into lines
        tweet_parts = tweet.split('\n')
        if len(tweet_parts) >= 3:
            date = tweet_parts[1]  # Assuming the date is always on the second line
            tweet_text = tweet_parts[2]  # Assuming the tweet text is always on the third line
            # Extract likes, retweets, etc. from the last line if present
            engagement = tweet_parts[-1] if len(tweet_parts) > 3 else ''
            likes, retweets, quote_tweets = '', '', ''
            if engagement:
                engagement_parts = engagement.split(' ')
                if len(engagement_parts) >= 3:
                    likes = engagement_parts[0]
                    retweets = engagement_parts[1]
                    quote_tweets = engagement_parts[2]
            writer.writerow([date, tweet_text, likes, retweets, quote_tweets])

print("Tweets saved to vitalik_tweets.csv")

# Close the WebDriver
driver.quit()

Tweets saved to vitalik_tweets.csv
